In [1]:
# https://www.youtube.com/watch?v=Vs730jsRgO8&t=285s&ab_channel=ElliotWaite
import os
import torch
from modelo_convolucional import instanciarModeloConvolucional

# mostrarle a onnx como debe ser el formato de entrada
dummy_input = torch.zeros(1,3,32,32)
config = {
        "batch_size": 16,
        "learning_rate": 0.000106647,
        "cant_filtros_conv1": 18,
        "kernel_size_maxpool1": 2,
        "cant_filtros_conv2": 28,
        "kernel_size_maxpool2": 3,
        "full_l1": 160,
        "full_l2": 104,
        "weight_decay":1e-4,
    }
modelo_conv = instanciarModeloConvolucional(config)
state_dict_path_conv = os.path.abspath("./copia_modelos_pre_produccion/modelo_checkpoint200_conv.pt")
modelo_conv.load_state_dict(torch.load(state_dict_path_conv))
# desactivar cualquier capa del modo de entrenamiento, onnx solo sirve para inferencia
modelo_conv.eval()
torch.onnx.export(modelo_conv,dummy_input, os.path.abspath("./produccion/model_conv.onnx"), verbose=True)


Exported graph: graph(%input.1 : Float(1, 3, 32, 32, strides=[3072, 1024, 32, 1], requires_grad=0, device=cpu),
      %layer1.0.weight : Float(18, 3, 5, 5, strides=[75, 25, 5, 1], requires_grad=1, device=cpu),
      %layer1.0.bias : Float(18, strides=[1], requires_grad=1, device=cpu),
      %layer2.0.weight : Float(28, 18, 5, 5, strides=[450, 25, 5, 1], requires_grad=1, device=cpu),
      %layer2.0.bias : Float(28, strides=[1], requires_grad=1, device=cpu),
      %denseLayer.1.weight : Float(160, 700, strides=[700, 1], requires_grad=1, device=cpu),
      %denseLayer.1.bias : Float(160, strides=[1], requires_grad=1, device=cpu),
      %denseLayer.4.weight : Float(104, 160, strides=[160, 1], requires_grad=1, device=cpu),
      %denseLayer.4.bias : Float(104, strides=[1], requires_grad=1, device=cpu),
      %denseLayer.6.weight : Float(10, 104, strides=[104, 1], requires_grad=1, device=cpu),
      %denseLayer.6.bias : Float(10, strides=[1], requires_grad=1, device=cpu)):
  %input : Float(

In [2]:
# lo mismo para MLP
from modelo_mlp import instanciarModeloMLP
modelo_mlp = instanciarModeloMLP(config)
state_dict_path_mlp = os.path.abspath("./copia_modelos_pre_produccion/modelo_checkpoint200_mlp.pt")
modelo_mlp.load_state_dict(torch.load(state_dict_path_mlp))
# desactivar cualquier capa del modo de entrenamiento, onnx solo sirve para inferencia
modelo_mlp.eval()
torch.onnx.export(modelo_mlp,dummy_input, os.path.abspath("./produccion/model_mlp.onnx"), verbose=True)


Exported graph: graph(%onnx::Flatten_0 : Float(1, 3, 32, 32, strides=[3072, 1024, 32, 1], requires_grad=0, device=cpu),
      %denseLayer.1.weight : Float(160, 3072, strides=[3072, 1], requires_grad=1, device=cpu),
      %denseLayer.1.bias : Float(160, strides=[1], requires_grad=1, device=cpu),
      %denseLayer.4.weight : Float(104, 160, strides=[160, 1], requires_grad=1, device=cpu),
      %denseLayer.4.bias : Float(104, strides=[1], requires_grad=1, device=cpu),
      %denseLayer.6.weight : Float(10, 104, strides=[104, 1], requires_grad=1, device=cpu),
      %denseLayer.6.bias : Float(10, strides=[1], requires_grad=1, device=cpu)):
  %onnx::Gemm_7 : Float(1, 3072, strides=[3072, 1], requires_grad=0, device=cpu) = onnx::Flatten[axis=1, onnx_name="Flatten_0"](%onnx::Flatten_0) # /home/braian/miniconda3/envs/tesis/lib/python3.9/site-packages/torch/nn/modules/flatten.py:45:0
  %input : Float(1, 160, strides=[160, 1], requires_grad=1, device=cpu) = onnx::Gemm[alpha=1., beta=1., transB=1,